<a href="https://colab.research.google.com/github/the-bucketless/iihf/blob/main/iihf_stats_and_lineups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests

In [2]:
def get_statistics_dfs(url):
    response = requests.get(url)
    try:
        dfs = pd.read_html(response.text)
    except ValueError:
        dfs = []

    if not len(dfs) > 1:
        return pd.DataFrame(), pd.DataFrame()

    team_skaters = [
        df
        .rename(columns={"j#": "jersey_number"})
        .drop(df.columns[[0, 3]], axis=1)
        .assign(team=dfs[0].teams.iloc[i], home=i == 0)
        for i, df in enumerate(dfs[1:6:4])
    ]

    for i in range(2):
        team_dfs = [dfs[2]] + dfs[10::4] if i == 0 else [dfs[6]] + dfs[12::4]
        for period, df in enumerate(team_dfs):
            if period > 0:
                df = (
                    df
                    .drop(columns="pos")
                    .add_suffix(f"_{period}")
                )

            team_skaters[i] = team_skaters[i].join(df)

    team_goalies = [
        name_df
        .rename(columns={"j#": "jersey_number"})
        .drop(name_df.columns[[0, 3]], axis=1)
        .assign(team=dfs[0].teams.iloc[i], home=i == 0)
        .join(
            stats_df
            .drop(columns=[c for c in stats_df.columns if "Unnamed" in c])
        )
        for i, (name_df, stats_df) in enumerate(zip(dfs[3:9:4], dfs[4:10:4]))
    ]

    skaters = pd.concat(team_skaters)
    goalies = pd.concat(team_goalies)

    skaters["statistics_url"] = url
    goalies["statistics_url"] = url

    return skaters, goalies

In [3]:
def get_lineups_df(url):
    response = requests.get(url)
    try:
        dfs = pd.read_html(response.text)
    except ValueError:
        dfs = []

    if not len(dfs) > 1:
        return pd.DataFrame()

    team_dfs = []
    n_team_dfs = len(dfs) // 2
    for i, df in enumerate(dfs[1:]):
        idx = i // n_team_dfs

        df = (
            df
            .drop(df.columns[0], axis=1)
            .rename(columns={"j#": "jersey_number"})
            .assign(
                line_number=i % n_team_dfs,
                team=dfs[0].teams.iloc[idx],
                home=idx == 0,
            )
        )

        if i > 0:
            df.columns = team_dfs[0].columns

        team_dfs.append(df)

    lineups = pd.concat(team_dfs)
    lineups["lineups_url"] = url

    return lineups

In [4]:
def get_game_dfs(game_url):
    skaters, goalies = get_statistics_dfs(game_url.replace("playbyplay", "statistics"))
    lineups = get_lineups_df(game_url.replace("playbyplay", "lineup"))

    if len(skaters):
        skaters["game_url"] = game_url
    if len(goalies):
        goalies["game_url"] = game_url
    if len(lineups):
        lineups["game_url"] = game_url

    return skaters, goalies, lineups

In [5]:
url = "https://www.iihf.com/en/events/2022/ww/gamecenter/playbyplay/37196/12-den-vs-hun"  #@param {type: "string"}

In [6]:
skaters, goalies, lineups = get_game_dfs(url)

In [13]:
skaters.iloc[:, :-2]

,jersey_number,name,team,home,pos,g,a,p,pim,ts,+/-,tot,shf,avg,ts_1,tot_1,ts_2,tot_2,ts_3,tot_3
0,14,JENSEN Nicoline,DEN,True,F,0,0,0,2,1,1.0,24:44,39.0,0:38,1,8:46,0,7:10,0,8:48
1,15,REFSGAARD Amanda,DEN,True,D,0,0,0,0,0,1.0,27:35,38.0,0:43,0,6:20,0,9:55,0,11:20
2,19,ASPERUP Josephine,DEN,True,D,0,0,0,0,1,0.0,26:17,38.0,0:41,0,7:58,0,9:27,1,8:52
3,21,WEIS Michelle,DEN,True,F,0,0,0,2,4,0.0,20:04,34.0,0:35,2,7:26,1,5:58,1,6:40
4,50,BAU Mia,DEN,True,F,0,0,0,0,1,0.0,10:21,17.0,0:36,1,4:08,0,2:15,0,3:58
5,4,GLUD Silke,DEN,True,F,0,0,0,4,4,1.0,16:37,23.0,0:43,1,7:21,2,4:10,1,5:06
6,8,PERSSON Josefine,DEN,True,F,0,0,0,2,1,1.0,22:23,32.0,0:41,1,6:52,0,6:17,0,9:14
7,22,SKOTT Sofie,DEN,True,D,0,0,0,0,0,0.0,19:03,26.0,0:43,0,6:48,0,5:47,0,6:28
8,63,JAKOBSEN Josefine,DEN,True,F,0,1,1,2,4,0.0,26:07,34.0,0:46,1,8:00,3,8:31,0,9:36
9,80,OSTERGAARD Julie,DEN,True,F,1,0,1,0,1,1.0,21:46,32.0,0:40,0,8:18,0,5:55,1,7:33


In [14]:
goalies.iloc[:, :-2]

,jersey_number,name,team,home,pos,sog,ga,svs,svs%
0,30,JENSEN Lisa,DEN,True,GK,45,0,45,100.00
1,72,REPSTOCK-ROMME Cassandra,DEN,True,GK,0,0,0,0.00
0,1,NEMETH Aniko,HUN,False,GK,19,1,18,94.74
1,33,REVESZ Zsuzsa,HUN,False,GK,0,0,0,0.00


In [15]:
lineups.iloc[:, :-2]

,jersey_number,Name,Pos,line_number,team,home
0,30.0,JENSEN Lisa,GK,0,DEN,True
1,72.0,REPSTOCK-ROMME Cassandra,GK,0,DEN,True
0,15.0,REFSGAARD Amanda,D,1,DEN,True
1,19.0,ASPERUP Josephine,D,1,DEN,True
2,21.0,WEIS Michelle,F,1,DEN,True
3,14.0,JENSEN Nicoline,F,1,DEN,True
4,50.0,BAU Mia,F,1,DEN,True
0,22.0,SKOTT Sofie,D,2,DEN,True
1,63.0,JAKOBSEN Josefine,F,2,DEN,True
2,4.0,GLUD Silke,F,2,DEN,True
